In [1]:
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

import rental_utils; reload(rental_utils)

%matplotlib inline
sns.set(style="whitegrid")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Philipp.Deutsch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Philipp.Deutsch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_raw = rental_utils.clean(pd.read_json('Data/train.json'))
test_raw = rental_utils.clean(pd.read_json('Data/test.json'))

In [3]:
train_descriptions = rental_utils.convert_to_words(train_raw)

 73% (36429 of 49352) |##############      | Elapsed Time: 0:01:21 ETA: 0:00:28C:\Users\Philipp.Deutsch\AppData\Local\Continuum\Anaconda2\envs\ds\lib\site-packages\bs4\__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
100% (49352 of 49352) |###################| Elapsed Time: 0:01:49 Time: 0:01:49


In [4]:
test_descriptions = rental_utils.convert_to_words(test_raw)

100% (74659 of 74659) |###################| Elapsed Time: 0:02:39 Time: 0:02:39


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer="word",
                                 tokenizer=None,
                                 preprocessor=None,
                                 stop_words=None,
                                 max_features=1000)
train_data_features = vectorizer.fit_transform(train_descriptions).toarray()

In [11]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 200) 
forest = forest.fit( train_data_features, train_raw["interest_level"].values)

predictions = forest.predict(train_data_features)
ct = pd.crosstab(predictions, train_raw["interest_level"].values, margins=False)
print(ct)

description_predictions = pd.DataFrame({'listing_id':train_raw['listing_id'], 'desc_pred':predictions})
description_predictions.to_csv('Data/description_prediction_train.csv', index=False)

col_0   high    low  medium
row_0                      
high    3128     72     204
low      316  33636    1440
medium   395    576    9585


In [12]:
test_data_features = vectorizer.transform(test_descriptions).toarray()

In [13]:
predictions = forest.predict(test_data_features)
description_predictions = pd.DataFrame({'listing_id':test_raw['listing_id'], 'desc_pred':predictions})
description_predictions.to_csv('Data/description_prediction_test.csv', index=False)

In [14]:
description_predictions['description_pred'].value_counts()

low       64288
medium     8124
high       2247
Name: description_pred, dtype: int64